# Column Transformer
- Column Transformer allows the application of different transformations to different subsets of features, enabling tailored preprocessing for various types of data.

In [1]:
import numpy as np
import pandas as pd


In [2]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [3]:
df = pd.read_csv('covid_toy.csv')

In [4]:
df.sample(5)

,age,gender,fever,cough,city,has_covid
5,84,Female,NaN,Mild,Bangalore,Yes
24,13,Female,100.0,Strong,Kolkata,No
42,27,Male,100.0,Mild,Delhi,Yes
28,16,Male,104.0,Mild,Kolkata,No
9,64,Female,101.0,Mild,Delhi,No


In [10]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [6]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(df.drop(columns = ['has_covid']),df['has_covid'],
                                                 test_size = 0.2)

In [9]:
x_train

,age,gender,fever,cough,city
55,81,Female,101.0,Mild,Mumbai
82,24,Male,98.0,Mild,Kolkata
56,71,Male,NaN,Strong,Kolkata
8,19,Female,100.0,Strong,Bangalore
65,69,Female,102.0,Mild,Bangalore
...,...,...,...,...,...
43,22,Female,99.0,Mild,Bangalore
59,6,Female,104.0,Mild,Kolkata
1,27,Male,100.0,Mild,Delhi
37,55,Male,100.0,Mild,Kolkata


## Without using Column Transformer
- More complex and time consuming 

In [11]:
## adding simple imputer to fever col

si = SimpleImputer()
x_train_fever = si.fit_transform(x_train[['fever']])

## also the test dataa
x_test_fever = si.fit_transform(x_test[['fever']])
x_train_fever.shape

(80, 1)

In [15]:
## Ordinal Encoding  ---> Cough

oe = OrdinalEncoder(categories = [['Mild','Strong']])
x_train_cough = oe.fit_transform(x_train[['cough']])

## also for the test part
x_test_cough = oe.fit_transform(x_test[['cough']])
                                 
x_train_cough.shape

(80, 1)

In [17]:
## OneHotEncoding ----> gender,city

ohe =  OneHotEncoder(drop = 'first',sparse_output = False)
x_train_gender_city = ohe.fit_transform(x_train[['gender','city']])

# for the test part
x_test_gender_city = ohe.fit_transform(x_test[['gender','city']])

x_train_gender_city.shape

(80, 4)

In [18]:
## Extracting Age from the column
x_train_age = x_train.drop(columns = ['gender','fever','cough','city']).values

# for the test part
x_test_age = x_test.drop(columns = ['gender','fever','cough','city']).values

x_train_age.shape

(80, 1)

In [20]:
## Concatinating all the columns after encoding

x_train_transformed = np.concatenate((x_train_age,x_train_fever,x_train_gender_city,x_train_cough),axis = 1)
x_train_transformed.shape

(80, 7)

In [21]:
# With the Help of ColumnTransformer

In [24]:
from sklearn.compose import ColumnTransformer

In [26]:
transformer = ColumnTransformer(
    transformers=[
        ('tnf1', SimpleImputer(), ['fever']),
        ('tnf2', OrdinalEncoder(categories=[['Mild', 'Strong']]), ['cough']),
        ('tnf3', OneHotEncoder(drop='first', sparse_output=False), ['gender', 'city'])
    ],
    remainder='passthrough'
)


In [28]:
transformer.fit_transform(x_train)

array([[101.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   1.        ,  81.        ],
       [ 98.        ,   0.        ,   1.        ,   0.        ,
          1.        ,   0.        ,  24.        ],
       [100.69444444,   1.        ,   1.        ,   0.        ,
          1.        ,   0.        ,  71.        ],
       [100.        ,   1.        ,   0.        ,   0.        ,
          0.        ,   0.        ,  19.        ],
       [102.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,  69.        ],
       [104.        ,   0.        ,   1.        ,   0.        ,
          1.        ,   0.        ,  16.        ],
       [102.        ,   0.        ,   1.        ,   0.        ,
          1.        ,   0.        ,   5.        ],
       [104.        ,   0.        ,   1.        ,   0.        ,
          0.        ,   0.        ,  51.        ],
       [101.        ,   1.        ,   0.        ,   1.        ,
          0.    

In [29]:
transformer.fit_transform(x_train).shape

(80, 7)

In [30]:
transformer.fit_transform(x_test).shape

(20, 7)